In [ ]:
# django with mysql needs mysqlclient package
# install mysqlclient - pip install mysqlclient

# create project
# issue the following command from command line
# django-admin startproject dbproject
# dbproject
#       dbproject
#       manage.py

# create application
# cd dbproject
# execute the following command
# python manage.py startapp dbapp ( this will create a directory named dbapp )

# setup the database
# create database
# create user

# update dbproject/settings.py and provide information for INSTALLED_APPS and DATABASES
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'dbapp'
]

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': 'demodb_2',
        'USER': 'demodb_user',
        'PASSWORD': 'password',
        'HOST': 'localhost',
        'PORT': '5432'
    }
}


# define appropriate model definitions in dbapp/models.py
from django.db import models

class Book(models.Model):
    booktitle = models.CharField(max_length=50)
    bookauthor = models.CharField(max_length=50)
    class Meta:
        db_table = 'books'
        
# convert the class definition to tables we need to run migration scripts
# execute the following from project directory
python manage.py makemigrations ( create necessary migration scripts)
python manage.py migrate  ( create tables based on migration scripts)


# create a file named forms.py below dbapp directory and add the following contents
from django import forms
from .models import Book

class BookForm(forms.ModelForm):
    class Meta:
        model = Book
        fields = "__all__"


# write appropritate view functions to fetch and add records

from django.shortcuts import render, redirect
from django.urls import reverse
from .models import Book
from .forms import BookForm

def books(request):
    books = Book.objects.all()
    return render(request,"dbapp/books.html",{'books': books})

def addbook(request):
    if request.method == 'POST':
        form = BookForm(request.POST)
        if form.is_valid():
            form.save()
            return redirect(reverse('dbapp:books'))
    else:
        form = BookForm()
        return render(request,"dbapp/addbook.html",{'form': form})
    
    
# create the template books.html below dbapp/templates/dbapp with the following contents
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
</head>
<body>
    <h1>List of Books</h1>
    <table>
        <th>Title</th>
        <th>Author</th>
        {% for book in books %}
        <tr>
            <td>{{ book.booktitle }}</td>
            <td>{{ book.bookauthor }}</td>
        </tr>
        {% endfor %}
    </table>
</body>
</html>

# create the template addbook.html below dbapp/templates/dbapp with the following contents
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
</head>
<body>
    <h1>Add New Book</h1>
    <form action="{% url 'dbapp:addbook' %}" method="post">
        {% csrf_token %} <!-- needed to prevent cross site scripting attack -->
        {{ form.as_p }}
        <input type="submit" value="submit">
    </form>
</body>
</html>

# create a file named urls.py below dbapp directory
from django.urls import path
from . import views

urlpatterns = [
    path('books',views.books,name='books'),
    path('addbook',views.addbook,name='addbook')
]

# update urls.py in dbproject
....
path('dbapp/',include(('dbapp.urls','dbapp'),namespace='dbapp')),
....
